# Stoichiometric Justification of Optimal KNSB Ratio

## Why this is important

We needed to determine an optimal grain geometry in terms of its dimensions and weight. This led us to solving first the equation for combustion of KNSB. This equation is given by:

$$
KNO_{3\left(s\right)}+C_6H_{14}O_{6\left(s\right)} \rightarrow CO_{2\left(g\right)}+CO_{\left(g\right)}+H_2O_{\left(g\right)}+H_{2\left(g\right)}+N_{2\left(g\right)}+K_2CO_{3\left(g\right)}+KOH_{\left(g\right)}
$$

As you have noticed, this chemical equation appears unbalanced. This is why we'll employ Python's power to this [task](https://tenor.com/v0Qj.gif).

## Let's get to business...

First, I'll set up the environment:

In [1]:
# This script installs the necessary packages from requirements.txt
%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


Then we'll import the required packages in our little script:

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
from sympy import symbols, Matrix

We'll then parse the formula by converting it into a 9x5 matrix.

> **A little heads up:**
> Such a matrix is generally difficult to solve using the normal methods since it's non-invertible. This is where we shall use *single value decomposition* (SVD). This is a function provided under `numpy.linalg`. This will be so cool!
> Scratch that, there's this neat python module called `sympy` that can solve it for us.

This is the matrix we'll be solving:

$$
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 1 \\
1 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 \\
3 & 6 & 2 & 1 & 1 & 0 & 0 & 3 & 1 \\
0 & 6 & 1 & 1 & 0 & 0 & 0 & 1 & 0 \\
0 & 14 & 0 & 0 & 2 & 2 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
a \\
b \\
c \\
d \\
e \\
f \\
g \\
h \\
i
\end{pmatrix}=
\begin{pmatrix}
0 \\
0 \\
0 \\
0 \\
0
\end{pmatrix}
$$

...where $a$ & $b$ are the reactants and $c$-$i$ are the products. You'll find that it's in the form of $Ax = b$. We'll parse the data in the `stoichiometric.csv` file and store it in variables `A` and `b` representing our $A$ and $b$, respectively:

In [3]:
df = pd.read_csv('../csv/stoichiometric.csv')
df.head()

print(df)

  Reactants  KNO3  C6H14O6  CO2  CO  H2O  H2  N2  K2CO3  KOH  b
0         K     1        0    0   0    0   0   0      2    1  0
1         N     1        0    0   0    0   0   2      0    0  0
2         O     3        6    2   1    1   0   0      3    1  0
3         C     0        6    1   1    0   0   0      1    0  0
4         H     0       14    0   0    2   2   0      0    1  0


In [4]:
A = Matrix(df.iloc[:, 1:-1].values)
b = Matrix(df.iloc[:, -1])

print(f"[INFO]: A: {A}")
print(f"[INFO]: b: {b}")

[INFO]: A: Matrix([[1, 0, 0, 0, 0, 0, 0, 2, 1], [1, 0, 0, 0, 0, 0, 2, 0, 0], [3, 6, 2, 1, 1, 0, 0, 3, 1], [0, 6, 1, 1, 0, 0, 0, 1, 0], [0, 14, 0, 0, 2, 2, 0, 0, 1]])
[INFO]: b: Matrix([[0], [0], [0], [0], [0]])


This route is pretty tedious. Let's use sympy to solve it. First, we shall abtain the analytical solution of the matrix $A$:

In [5]:
analytical_sol = None
analytical_sol = A.gauss_jordan_solve(b, freevar=True)

print(f"[INFO]: analytical solution: {analytical_sol}")

[INFO]: analytical solution: (Matrix([
[                           -2*tau2 - tau3],
[               -tau0/7 - tau1/7 - tau3/14],
[                  -tau0 + 4*tau2 + 2*tau3],
[13*tau0/7 + 6*tau1/7 - 5*tau2 - 11*tau3/7],
[                                     tau0],
[                                     tau1],
[                            tau2 + tau3/2],
[                                     tau2],
[                                     tau3]]), Matrix([
[tau0],
[tau1],
[tau2],
[tau3]]), [4, 5, 7, 8])


Then, we shall obtain a numerical solution at $\tau_n=1$ as follows: 

In [6]:
tau0, tau1, tau2, tau3 = analytical_sol[1]

numerical_sol = analytical_sol[0].subs({tau0: 1, tau1: 1, tau2: 1, tau3: 1})
print(f"[INFO]: numerical solution: {numerical_sol}")

[INFO]: numerical solution: Matrix([[-3], [-5/14], [5], [-27/7], [1], [1], [3/2], [1], [1]])


As you might've noticed, the numerical solution at $\tau_m = 1$ is full of negative values. We require the values of $\tau_0$, $\tau_1$, $\tau_2$ and $\tau_3$ to yield a positive result. For this we'll need `scipy.linprog` to obtain a feasible or error out.

First we'll convert `analytical_sol[0]` to an $m\times n$ matrix where $m=4$ and $n=9$ to be our **unbounded** $A$ or $A_{ub}$ and then define a vector of small negative margins, **epsilon** or our **unbounded** $b$ ($b_{ub}$), to ensure strict positivity, as follows:

In [7]:
e = 1e-6                            # epsilon
A_ub = [
    [0, 0, 2, 1],                    # 2*tau2 + tau3 < 0 (from -2*tau2 - tau3 > 0)
    [1, 1, 0, 0.5],                  # tau0 + tau1 + tau3/2 < 0
    [1, 0, -4, -2],                  # tau0 - 4*tau2 - 2*tau3 < 0
    [-13/7, -6/7, 5, 11/7],          # -13*tau0/7 - 6*tau1/7 + 5*tau2 + 11*tau3/7 < 0
    [-1, 0, 0, 0],                   # -tau0 < 0 (tau0 > 0)
    [0, -1, 0, 0],                   # -tau1 < 0 (tau1 > 0)
    [0, 0, -1, -0.5],                # -tau2 - tau3/2 < 0
    [0, 0, -1, 0],                   # -tau2 < 0 (tau2 > 0)
    [0, 0, 0, -1]                    # -tau3 < 0 (tau3 > 0)
]
b_ub = [-e] * 9
dummy = [0] * 4
bounds = [(e, None)] * 4

You'll notice that this will be reducing our solution to a linear programming problem. This will be like this:

In [8]:
result = sp.optimize.linprog(c=dummy, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

if result.success:
    tau_val = result.x
    print(f"[INFO]: optimal tau values: {tau_val}")
else:
    print(f"[ERROR]: no feaible solution found: {result.message}")
    exit(-2)

[ERROR]: no feaible solution found: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)


Since we've proven that the above chemical reaction is unbalanceable, we'll simplify it to the following:

$$
a\: KNO_{3\left(s\right)}+b\: C_6H_{14}O_{6\left(s\right)} \rightarrow c\: CO_{2\left(g\right)}+d\: H_2O_{\left(g\right)}+e\: N_{2\left(g\right)}+f\: K_2CO_{3\left(l\right)}+\Delta H^\theta_c
$$

which reduces the matrix to a 6x5 as follows:

$$
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 2 \\
1 & 0 & 0 & 0 & 2 & 0 \\
3 & 6 & 2 & 1 & 0 & 3 \\
0 & 6 & 1 & 1 & 0 & 1 \\
0 & 14 & 0 & 0 & 0 & 0
\end{pmatrix}
\begin{pmatrix}
a \\
b \\
c \\
d \\
e \\
f
\end{pmatrix}
=
\begin{pmatrix}
0 \\
0 \\
0 \\
0 \\
0
\end{pmatrix}
$$

adjusting this from our `stoichiometric.csv` file we get:

In [9]:
A1 = Matrix(df.iloc[:, [1, 2, 3, 5, 7, 8]].values)
b1 = Matrix(df.iloc[:, -1].values)

analytical_sol1 = A1.gauss_jordan_solve(b1, freevar=True)
tau0 = analytical_sol1[1][0]
numerical_sol1 = analytical_sol1[0].subs({tau0: 1})

print(f"[INFO]: numerical solution 1: {numerical_sol1}")

[INFO]: numerical solution 1: Matrix([[-2], [-5/13], [17/13], [35/13], [1], [1]])


## Determining the Ideal Mass Ratio

From this, we can deduce that based on the $O_2$ **demand** (*the amount of $O_2$ needed to completely combust $C_6H_{14}O_6$*)  by the following equation:

$$
C_6H_14O_{6\left(s\right)}+O_{2\left(g\right)} \rightarrow 6CO_{2\left(g\right)}+7H_2O_{\left(g\right)}
$$

we'll need 6 $mol\: O_2$ per $mol\: C_6H_{14}O_6$. The amount of $O_2$ provided by $KNO_3$ will be 3 $O$ atoms or 1.5 $mol\: O_2$. Thus, you'll need:

$$
\frac{6}{1.5}=4\: mol\: KNO_3\: per\: mol\: C_6H_{14}O_6
$$

Based on the molar masses of $KNO_3=101.10320\: g/mol$ and $C_6H_{14}O_6=182.17\: g/mol$ i.e , we can deduce that:

$$
For\: 1mol\: C_6H_{14}O_6,\: we'll\: need\: 4 \times 101.10320 = 404.41280\: g\: KNO_3
$$

We'll therefore have the following ideal mass ratio:

$$
C_6H_{14}O_6:KNO_3 = \frac{M_{C_6H_{14}O_6}}{M_{KNSB}} =\frac{404.4124}{404.4124+182.17} = 0.6836 \\
$$

In terms of percentages, $68.36\%:31.64\%$ is the most ideal ratio.

## Why would we go below the stoichiometric ratio?

It's been proven experimentally that slightly fuel rich ratios i.e. $65\%:35\%$ confer the following advantages:

- Flame temperatures that are below **Adiabatic Flame Temperature**: thus less stress on the motor casing
- More gaseous products e.g. $CO_2$, $N_2$ and $H_2$: this has a positive effect on the resultant $I_{sp}$
- Slight oxygen deficiency that limits formation of $K_2CO_3$ slag: reducing nozzle clogging
- More $C_6H_{14}O_6$ makes casting process easier due to a more binded slurry

Due to these thermochemical properties, we encourage the tuning of the propellant mix. ([Nakka, 2025](https://www.nakka-rocketry.net/sorb.html))

## So, in conclusion...

While the stoichiometric $O/F$ mass ratio for complete combustion of $C_6H_{14}O_6$ using $KNO_3$ is approximately $68\%:32\%$, the $65\%:35\%$ mixture is chosen to slightly fuel-bias the composition. This helps reduce chamber temperature, minimize slag production, and increase the specific impulse through favorable gas-phase species like $CO$ and $H_2$. This near-stoichiometric yet fuel-rich blend results in an efficient, castable, and safer propellant configuration.

## References

1. R. Nakka, "KNSB Propellant," *Richard Nakka's Experimental Rocketry Web Site*, 2025. [Online]. Available: https://www.nakka-rocketry.net/sorb.html
2. J. Bonnie, J. Zehe, and S. Gordon, *NASA Glenn Coefficients for Calculating Thermodynamic Properties of Individual Species*, NASA/TP—2002-211556, Glenn Research Center, Cleveland, 2002.